# Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Импорт предобработанных данных

In [ ]:
!git clone https://github.com/usmaxnik-rgb/VKR_UMN

In [ ]:
X_united_full=pd.read_excel('/content/VKR_UMN/X_united_full.xlsx', index_col='Index')

# Разбиение на тренировочную и тестовую выборки

In [ ]:
y=np.array(X_united_full['Соотношение матрица-наполнитель'])
X=np.array(X_united_full.drop('Соотношение матрица-наполнитель', axis=1))

X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, shuffle=True, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

# Нормализация данных

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создание модели нейронной сети

In [ ]:
model = Sequential()


# Входной слой + первый скрытый
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.3))


# Второй скрытый слой
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))


# Третий скрытый слой
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))


# Выходной слой (регрессия)
model.add(Dense(1, activation='linear'))


# Компиляция модели
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mean_squared_error',
    metrics=['mae']
)

# Обучение модели с ранней остановкой

In [ ]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

history = model.fit(
    X_train_scaled, y_train,
    epochs=200,
    batch_size=16,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1
)


# Предсказание на тестовой выборке

In [ ]:
y_pred = model.predict(X_test_scaled).flatten()

# Расчёт метрик качества
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)


# Вывод результатов
print("Метрики качества модели:")
print(f"MAE (Средняя абсолютная ошибка):      {mae:.4f}")
print(f"MSE (Средняя квадратичная ошибка):       {mse:.4f}")
print(f"RMSE (Корень из MSE):                  {rmse:.4f}")
print(f"R² (Коэффициент детерминации):         {r2:.4f}")


# Визуализация: история обучения
plt.figure(figsize=(12, 4))


plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()


# Визуализация: предсказания vs истинные значения
plt.subplot(1, 2, 2)
plt.scatter(y_test, y_pred, alpha=0.6)
plt.xlabel('True Values')
plt.ylabel('Predictions')
plt.title('Predictions vs True Values')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.show()

# Сохранение модели

In [ ]:
model.save('tuned_model.keras')
print("Обученная модель сохранена как 'tuned_model.keras'")